In [19]:
import gymnasium as gym
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import seaborn as sns
from tqdm import tqdm
from collections import defaultdict # allows access to undefined keys
matplotlib.use('TkAgg')  # or 'Qt5Agg' if you prefer Qt

In [20]:
class PongAgent:
    def __init__(self,
                 learning_rate: float,
                 initial_epsilon: float,
                 epsilon_decay: float,
                 final_epsilon: float,
                 discount_factor: float = 0.95,
                 discrete_actions: int = 6):
        
        self.lr = learning_rate
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.discount_factor = discount_factor
        self.discrete_actions = discrete_actions
        
        # Initialize Q-table
        self.q_values = defaultdict(lambda: np.zeros(self.discrete_actions))
        
        self.training_error = []
    
    def discretize_state(self, state):
        # Round each value in the state to 1 decimal place
        # Convert to tuple for hashability
       
          # Slice to exclude last element
        # Append the original terminated flag (boolean)
        return tuple(np.append(state, state[-1])) 
    
    def choose_action(self, state):
        discretized_state = self.discretize_state(state)
        
        if np.random.random() < self.epsilon:
            return np.random.randint(self.discrete_actions)
        else:
            return int(np.argmax(self.q_values[discretized_state]))
    
    def update_q_values(self, state, action, reward, terminated, next_state):
        state = self.discretize_state(state)
        next_state = self.discretize_state(next_state)
        
        if not terminated:          
            future_q_value = np.max(self.q_values[next_state])
        else:
            future_q_value = 0
        temporal_difference = (reward + (self.discount_factor * future_q_value)) - self.q_values[state][action]
        self.q_values[state][action] += self.lr * temporal_difference
        self.training_error.append(temporal_difference)
        
    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon * self.epsilon_decay)

In [21]:
learning_rate = .1
n_episodes = 10_000
start_epsilon = 1
epsilon_decay = 0.99
final_epsilon = 0.05

agent = PongAgent(
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    final_epsilon=final_epsilon,
    epsilon_decay=epsilon_decay,
    
    
)

In [22]:
env = gym.make("Pong-ramNoFrameskip-v0", render_mode='rgb_array')


In [23]:
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
env = gym.wrappers.TimeLimit(env, max_episode_steps=60)

rewards = 0 
for episode in tqdm(range(n_episodes)):
    obs, info = env.reset()
    done = False
    # play one episode
    while not done:
        action = agent.choose_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        rewards += reward
        # update the agent
        agent.update_q_values(obs, action, reward, terminated, next_obs)

        # update if the environment is done and the current obs
        done = terminated or truncated
        obs = next_obs

    agent.decay_epsilon()

100%|██████████| 10000/10000 [03:03<00:00, 54.48it/s]


In [24]:
q_values = np.array([value for key, value in agent.q_values.items()])
print(np.argmax(q_values,axis=1))


[0 0 0 ... 0 0 0]


In [25]:
print(f'total rewards = {rewards}')


total rewards = 0.0


In [26]:

# Create and wrap the environment
env = gym.make("Pong-ramNoFrameskip-v0",render_mode='rgb_array')
# env = CustomRewardWrapper(env)

obs, info = env.reset()

plt.ion()
fig, ax = plt.subplots(figsize=(8,8))
action_text = ax.text(510, 20, '', color='white', fontsize=12, bbox=dict(facecolor='blue', alpha=0.8))
img = ax.imshow(env.render())
actions = ['Move Up','Move Right','Move Down','Move Left']
rewards = 0
num_epochs= 1
for step in range(num_epochs):
    obs, info = env.reset()
    done = False
    print(f'step {step}:  obs = {next_obs} , reward = {reward}')
    while not done:
        action = agent.choose_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        rewards += reward
        if reward < 0 : 
            print(f'step {step}:  obs = {next_obs} , reward = {reward}')

        frame = env.render()
        img.set_data(frame)
        action_text.set_text(f'Step: {[action] }')

        fig.canvas.draw()
        fig.canvas.flush_events()
        done = terminated or truncated
        obs = next_obs

plt.ioff()  # Turn off interactive mode
plt.show()  # Keep the window open after the animation finishes
plt.close()
env.close()

C:\Users\Click\PycharmProjects\DeepLearning\.venv\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


step 0:  obs = [192   0   0   0 110  38   0   7 131   1   0   0  70   0   0  63 255   0
 255 253   0 130   0  24 128  32   1  86 247  86 247  86 247 134 243 245
 243 240 240 242 242  32  32  64  64  64 188  65 189 126 130 109  37  37
 130   0   1   0   1 109 109  37  37 192 192 192 192   1 192 202 247 202
 247 202 247 202 247   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  53  85  54 236 242
 121 240] , reward = 0.0
step 0:  obs = [192   0   0   0 110  38   0   7  71   2   0  63  14   1   0  63 255   1
   0   2   0 162   0  24 128  32   1  86 247  86 247  86 247 134 243 245
 243 240 240 242 242  32  32  64  64  64 188  65 189 205 162  67  37  37
 175   0   2   0 255 109  67  37  37 192 192 192 192 192 192 207 247 202
 247 202 247 202 247   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

TclError: can't invoke "update" command: application has been destroyed

In [ ]:
print(f'mean episode rewards = {rewards/num_epochs}')

In [ ]:
print(f'action space shape : {env.action_space.n}') # Number of possible actions is 4
print(f'observation space shape : {env.observation_space}') 
# print(f'observation space numbers : {env.nS}') 
#-------------- obesrvation is a tupe of 3 values : --------------
#1) player cards value
#2) dealer's face up card
#3) usable ace for player, equal 1 if ace is considered an 11 without busting

print(f'reward range : {env.reward_range}') # default reward range is set to -inf +inf
# print(f'\nEnv specs : {env.spec}') 
print(f'\nEnv metadata : {env.metadata}') # render_modes adn render_fps